In [4]:
import pandas as pd
import numpy as np
from scripts.preprocess import ts_preprocess, scale
import json
from scripts.model import CNN_LSTM, LightningLSTM
import torch
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix

In [5]:
point = 57

#load hyper params
with open(f'scripts/best_params/{point}.json') as f:
    best = json.load(f)
best = best['best_params']

dataframe = pd.read_csv(f'datos_prueba/datos_prueba.csv')
dataframe = dataframe.loc[dataframe["id"] == point]
#convertimos a formato de datetime
dataframe["datetime"] = pd.to_datetime(dataframe["datetime"])
df_sorted = dataframe.sort_values(by=["datetime"])
dataframe_fs = df_sorted.drop(['feels_like', 'icon', 'id'], axis=1)
dataframe_fs["datetime"] = dataframe_fs["datetime"].dt.hour
dataframe_fs["rain_prob"] = (dataframe_fs['rain_prob'] >= 0.5).astype(int)
X = dataframe_fs.to_numpy()

scale_method = best.pop('scale_method')
n_steps = best.pop('n_steps')

#normalizamos los datos
X = scale(X, scale_method)
#convertimos en en shape (N, seq_length, n_columns)
X = ts_preprocess(X, n_steps=n_steps)
X.shape

(10, 39, 11)

In [6]:
#load model
if best['model_name'] == 'cnn_lstm':
    model = CNN_LSTM(X.shape[2], hidden_size=best['hidden_size'], num_layers=best['num_layers'], dropout=best['dropout'], kernel_size=best['kernel_size'], num_cnn=best['num_cnn'])
elif best['model_name'] == 'lstm':
    model = LightningLSTM(X.shape[2], hidden_size=best['hidden_size'], num_layers=best['num_layers'], dropout=best['dropout'])
else:
    raise Exception
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = torch.load(f'trained_models/{point}.pth',
                        map_location=torch.device(device),
                        weights_only=True)

model.load_state_dict(checkpoint)
model.eval()

#predecimos
y_pred = model(torch.tensor(X, dtype=torch.float32)).cpu().detach().numpy()
y_pred = y_pred.reshape(y_pred.shape[0])
y_pred_logit = y_pred.copy()

optimal_threshold = 0.5
y_pred[ y_pred > optimal_threshold] = 1
y_pred[ y_pred <= optimal_threshold] = 0

#mostramos 30 primeros
print(f"preditions logits:\n{y_pred_logit[:30]}")
print(f"preditions:\n{y_pred[:30].astype(np.int32)}")

Seed set to 42


preditions logits:
[9.9998295e-01 9.3134731e-01 3.5595307e-01 6.0166017e-04 6.5908622e-05
 1.0449707e-03 2.5802511e-01 9.6963221e-01 9.9880564e-01 5.9429407e-01]
preditions:
[1 1 0 0 0 0 0 1 1 1]
